In [2]:
import numpy as np
from matplotlib.image import imread


class ImageDatasetBuilder:
    """
    Given a DataFrame whose index is a set of image IDs (as with {train, test}.csv), returns featurized images.
    """
    def __init__(self, x_dim=100, y_dim=100, source='../data/train/images/', mask=False):
        """
        Builds the featurized image transform.
        
        x_dim: int
            The X dimension to crop the images to.
        y_dim: int
            The Y dimension to crop the images to.
        source: str
            Path to the folder containing the image files.
        mask: booleon
            If true, the underlying data is a mask. If false, the underlying data is RGB. If the data is RGB,
            we take just the R component and skip the GB, because the images are grayscale anyway.
        """
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.source = source
        self.mask = mask
        
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        r = np.asarray(
            list(
                map(lambda img_id: np.ravel(
                    imread(f'{self.source}/{img_id}.png')[:self.x_dim,:self.y_dim]
                ), X.index.values)
            )
        )
        return r if self.mask else r[:,::3]
    
    
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier


def buildClassifier(dim=50, hidden_nodes=100):
    """Returns the Keras model."""
    def ret():
        clf = Sequential()
        clf.add(Dense(hidden_nodes, activation='relu', input_dim=dim**2))
        clf.add(Dense(dim**2, activation='sigmoid'))
        clf.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.1), metrics=['accuracy'])
        return clf
    return ret

Using TensorFlow backend.


TODO: outputting to hugely-dimensioned output target.

In [4]:
import pandas as pd
train = pd.read_csv("../data/train.csv", index_col="id", usecols=[0])
_depths = pd.read_csv("../data/depths.csv", index_col="id")
train = train.join(_depths)
test = _depths[~_depths.index.isin(train.index)]
del _depths

X = ImageDatasetBuilder(source='../data/train/images/').transform(train)
y = ImageDatasetBuilder(source='../data/train/masks/', mask=True).transform(train)

In [23]:
from keras.metrics import binary_accuracy

clf = Sequential()
clf.add(Dense(100**2, activation='relu', input_dim=100**2))
clf.add(Dense(100, activation='relu', input_dim=100**2))
clf.add(Dense(4, activation='sigmoid'))
clf.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01), metrics=[binary_accuracy])

In [25]:
clf.fit(X, y[:, :4], epochs=20, batch_size=1000)

Epoch 1/20
4000/4000 [==============================] - 10s 3ms/step - loss: 1.2021 - binary_accuracy: 0.2191
Epoch 2/20
4000/4000 [==============================] - 10s 2ms/step - loss: 1.2021 - binary_accuracy: 0.2191
Epoch 3/20
4000/4000 [==============================] - 10s 2ms/step - loss: 1.2021 - binary_accuracy: 0.2167
Epoch 4/20
4000/4000 [==============================] - 10s 2ms/step - loss: 1.2020 - binary_accuracy: 0.2167
Epoch 5/20
4000/4000 [==============================] - 10s 2ms/step - loss: 1.2020 - binary_accuracy: 0.2167
Epoch 6/20


KeyboardInterrupt: 

In [16]:
clf.predict(X[:100])

array([[0.6171473 , 0.55348605, 0.55309147, 0.61769736],
       [0.5543055 , 0.5378089 , 0.63753796, 0.55272293],
       [0.57653797, 0.5550737 , 0.5671964 , 0.5574952 ],
       [0.5674835 , 0.5725575 , 0.5688045 , 0.48926324],
       [0.5817462 , 0.5701564 , 0.58210295, 0.6019498 ],
       [0.6481027 , 0.65265006, 0.622256  , 0.6350627 ],
       [0.5931024 , 0.5812331 , 0.6193448 , 0.618674  ],
       [0.5800271 , 0.5958613 , 0.61456597, 0.602808  ],
       [0.588117  , 0.5374546 , 0.6476319 , 0.59946835],
       [0.5958082 , 0.597879  , 0.5787285 , 0.58003324],
       [0.6053374 , 0.5934818 , 0.6162661 , 0.5386595 ],
       [0.5522847 , 0.5891118 , 0.5886695 , 0.57521176],
       [0.63978374, 0.5982208 , 0.55435365, 0.5823599 ],
       [0.55425   , 0.58499396, 0.5994906 , 0.62034017],
       [0.5995822 , 0.618214  , 0.6087585 , 0.5596142 ],
       [0.5667329 , 0.5400737 , 0.5634217 , 0.55614567],
       [0.6490746 , 0.6174215 , 0.6011948 , 0.6548847 ],
       [0.62814146, 0.58637244,

In [25]:
# from sklearn.pipeline import make_pipeline

# clf = make_pipeline(
#     KerasClassifier(buildClassifier(dim=100, hidden_nodes=100), epochs=2, batch_size=20)
# )

In [26]:
# clf.fit(X, y)

In [27]:
# from sklearn.model_selection import KFold
# from sklearn.metrics import accuracy_score

# kf = KFold(n_splits=4)

# scores = []
# for train_index, test_index in kf.split(X):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
    
#     scores.append(
#         accuracy_score(clf.fit(X_train, y_train).predict(X_test), y_test)
#     )
    
# np.mean(scores)